In [1]:
# important modules for this HW
import bs4 # doc: https://www.crummy.com/software/BeautifulSoup/bs4/doc/
import requests as rq 
import re
import time
# previous useful modules
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context('notebook')

In [57]:
import copy

# Loading the 2 dataset

In [2]:
df1=pd.read_csv('/home/stefano/Dropbox/Universita/Losanna/Exam/Anno2/DataAnalysis/main_database_1.csv')

In [3]:
df2=pd.read_csv('/home/stefano/Dropbox/Universita/Losanna/Exam/Anno2/DataAnalysis/main_database_2.csv')

In [4]:
df=pd.concat([df1,df2],axis=0)

In [5]:
df.head()

,cat,sex,rang,nom,an,lieu,temps,retard,pace,link
0,M/M30,M,16,Abderhalden Seth,69,Luzern,"3:33.32,9","37.38,2",False,http://services.datasport.com/1999/lauf/biel/A...
1,Klasse8,False,8,"Abdirizak Ayanle, Biel/Bienne",84,Schule Sahligut Hinterman,"14.02,3","5.06,5",False,http://services.datasport.com/1999/lauf/biel/A...
2,Klasse8,False,3,"Abduli Ajete, Biel/Bienne",84,Schule Sahligut Hinterman,"10.50,4","1.54,6",False,http://services.datasport.com/1999/lauf/biel/A...
3,Klasse8,False,9,"Abou Al Nassr Chaled, Biel/Bienne",84,Schule Sahligut Hinterman,"14.09,9","5.14,1",False,http://services.datasport.com/1999/lauf/biel/A...
4,H/M35,M,1,Abplanalp Michael,64,Bern,"1:16.53,8",---,False,http://services.datasport.com/1999/lauf/biel/A...


In [6]:
df['index'] = np.arange(len(df))
df = df.set_index('index')

# Links2run integration

### Loading links2run

In [7]:
links2runs=pd.read_csv('../datasets/links2runs.csv')
del links2runs['Unnamed: 0']

In [8]:
links2runs=links2runs.set_index(links2runs.URL)

In [9]:
links2runs.head(3)

,Date,Name,URL
URL,,,
http://services.datasport.com/1999/zkb/maennedorf,sam. 27.03.1999,Männedörfler Waldlauf,http://services.datasport.com/1999/zkb/maennedorf
http://services.datasport.com/1999/lauf/kerzers,sam. 20.03.1999,Kerzerslauf,http://services.datasport.com/1999/lauf/kerzers
http://services.datasport.com/1999/lauf/luzern,sam. 24.04.1999,Luzerner Stadtlauf,http://services.datasport.com/1999/lauf/luzern


In [10]:
links2runs.loc['http://services.datasport.com/2000/lauf/tdcn']

,Date,Name,URL
URL,,,
http://services.datasport.com/2000/lauf/tdcn,mer. 26.04.2000,BCN Tour St. Aubin,http://services.datasport.com/2000/lauf/tdcn
http://services.datasport.com/2000/lauf/tdcn,mer. 19.04.2000,BCN Tour Le Landeron,http://services.datasport.com/2000/lauf/tdcn
http://services.datasport.com/2000/lauf/tdcn,mer. 24.05.2000,BCN Tour Neuchâtel,http://services.datasport.com/2000/lauf/tdcn
http://services.datasport.com/2000/lauf/tdcn,mer. 17.05.2000,BCN Tour La Sagne,http://services.datasport.com/2000/lauf/tdcn
http://services.datasport.com/2000/lauf/tdcn,mer. 10.05.2000,BCN Tour Dombresson,http://services.datasport.com/2000/lauf/tdcn
http://services.datasport.com/2000/lauf/tdcn,mer. 03.05.2000,BCN Tour Noiraigue,http://services.datasport.com/2000/lauf/tdcn


In [11]:
links2runs.index.value_counts()

http://services.datasport.com/2000/lauf/tdcn                      6
http://services.datasport.com/2011/lauf/tdpv/etape4               2
http://services.datasport.com/2002/lauf/tdpv/etape3               2
http://services.datasport.com/2006/lauf/morat                     2
http://services.datasport.com/2003/lauf/tdpv/etape3               2
http://services.datasport.com/2010/lauf/tdpv/etape4               2
http://services.datasport.com/2007/lauf/zueriwalkt                2
http://services.datasport.com/2007/diverse/tzampata               2
http://services.datasport.com/2009/lauf/tdpv/etape4               2
http://services.datasport.com/2009/lauf/defi                      2
http://services.datasport.com/2011/lauf/gempen                    1
http://services.datasport.com/2006/lauf/gurten                    1
http://services.datasport.com/2010/lauf/tdpv/etape3               1
http://services.datasport.com/2012/lauf/arosa                     1
http://services.datasport.com/1999/lauf/eiken   

### Process link

In [12]:
df.link=df.link.apply(lambda x: '/'.join(x.split('/')[:-1]))

In [13]:
df.head(2)

,cat,sex,rang,nom,an,lieu,temps,retard,pace,link
index,,,,,,,,,,
0,M/M30,M,16,Abderhalden Seth,69,Luzern,"3:33.32,9","37.38,2",False,http://services.datasport.com/1999/lauf/biel
1,Klasse8,False,8,"Abdirizak Ayanle, Biel/Bienne",84,Schule Sahligut Hinterman,"14.02,3","5.06,5",False,http://services.datasport.com/1999/lauf/biel


### Groupby link

In [14]:
links=list(df.link.value_counts().index)

In [15]:
def add_date_name_to_group(x):
    link=x.link.iloc[0]
#     print(link)
#     print('\n\n\n\n')
    row_links2runs=links2runs.loc[link]
#     print(row_links2runs.shape)
    if row_links2runs.shape==(2,3):
        row_links2runs=row_links2runs.iloc[0]
#     print(row_links2runs.shape)
    x['Date']=row_links2runs.Date
    x['Name']=row_links2runs.Name
    return x

In [16]:
grouped_link=df.groupby(df.link)

In [17]:
df=grouped_link.apply(add_date_name_to_group)

In [18]:
del df['link']

In [19]:
df.head(2)

,cat,sex,rang,nom,an,lieu,temps,retard,pace,Date,Name
index,,,,,,,,,,,
0,M/M30,M,16,Abderhalden Seth,69,Luzern,"3:33.32,9","37.38,2",False,ven. 11.06.1999,"Bieler Lauftage, Biel/Bienne"
1,Klasse8,False,8,"Abdirizak Ayanle, Biel/Bienne",84,Schule Sahligut Hinterman,"14.02,3","5.06,5",False,ven. 11.06.1999,"Bieler Lauftage, Biel/Bienne"


# Delete duplicates
Some links of games are counted twice. In this case we check for duplicated rows and we remove them

In [20]:
df=df.drop_duplicates(keep='first')

# Process temps-retards

In [21]:
class Time:
    def __init__(self,time,mseconds=True):
        self.hours,self.minutes,self.seconds,self.mseconds=self.parse_time(time,split=True,mseconds=mseconds)
        
    def parse_time(self,time,check_only=False,split=True,mseconds=True):
        ''' Return a parsing of the time
        '''
        try :
            time.hours
            return (time.hours,time.minutes,time.seconds,time.mseconds)
        except:
            pass
        if split:
            time=time.split(' ')[0]
        if time.count(',')==0 and not check_only and mseconds:
            raise()
        time=re.split("[:.,]+",time)
        if not mseconds:
            time=time+[0]
        while len(time)<4:
            time=[0]+time
        hours,minutes,seconds,mseconds=[float(x) for x in time]

        if not check_only:
            return (hours,minutes,seconds,mseconds)
        
    def get_seconds(self):  
        return self.hours*3600 + self.minutes*60 + self.seconds + self.mseconds/1000
    
    def __str__(self):
        return str(int(self.hours))+':'+str(int(self.minutes))+'.'+str(int(self.seconds))+','+str(int(self.mseconds))
    
    def __div__(self,other):
        try:
            seconds_other=other.get_seconds()
        except:
            return False
        return self.get_seconds()-seconds_other
    
    def __truediv__(self,other):
        try:
            seconds_other=other.get_seconds()
        except:
            return False
        return self.get_seconds()/seconds_other
    

In [22]:
dict_problems={}

In [23]:
re.split("[:.]+",'14.46')

['14', '46']

In [24]:
def apply_time(x,mseconds=True):
    try:
        time=Time(x,mseconds=mseconds)
    except:
        try:
            dict_problems[x]+=1
        except:
            dict_problems[x]=1
        return False
    return time

In [25]:
df.temps=df.temps.apply(apply_time)

In [26]:
df.retard=df.retard.apply(apply_time)

In [27]:
df.pace=df.pace.apply(apply_time,args=(False,))

In [28]:
a=Time('48.12,2')
b=Time('4.30,0')
print(a,b)

0:48.12,2 0:4.30,0


In [29]:
b.get_seconds()

270.0

In [30]:
a.get_seconds()

2892.002

In [31]:
a/b

10.711118518518518

In [32]:
df['Distance']=df.temps/df.pace

In [33]:
df.head(2)

,cat,sex,rang,nom,an,lieu,temps,retard,pace,Date,Name,Distance
index,,,,,,,,,,,,
0,M/M30,M,16,Abderhalden Seth,69,Luzern,"3:33.32,9","0:37.38,2",False,ven. 11.06.1999,"Bieler Lauftage, Biel/Bienne",False
1,Klasse8,False,8,"Abdirizak Ayanle, Biel/Bienne",84,Schule Sahligut Hinterman,"0:14.2,3","0:5.6,5",False,ven. 11.06.1999,"Bieler Lauftage, Biel/Bienne",False


In [34]:
df_old=df.copy()

In [35]:
df=df[df.Distance>0]

In [36]:
df_old.shape

(3007467, 12)

In [276]:
df=df_old[df_old.Distance>0]

# Sex Male-Female

### Get names database

In [277]:
df.shape

(2344537, 12)

In [278]:
df.head(5)

,cat,sex,rang,nom,an,lieu,temps,retard,pace,Date,Name,Distance
index,,,,,,,,,,,,
3027,M20,M,233,Abgottspon Peter,74,Zermatt,"1:2.25,0",False,"0:4.9,0",sam. 18.03.2000,Kerzerslauf,15.0402
3028,M35,M,32,Abplanalp Michael,64,Bern,"0:55.11,7",False,"0:3.40,0",sam. 18.03.2000,Kerzerslauf,15.05
3029,M50,M,155,Abt Werner,47,Spiez,"1:12.42,9",False,"0:4.50,0",sam. 18.03.2000,Kerzerslauf,15.0414
3030,F45,F,48,Ackermann Antoinette,53,Alterswil,"1:22.36,7",False,"0:5.30,0",sam. 18.03.2000,Kerzerslauf,15.0182
3031,F50,F,42,Ackermann Hedy,46,Alterswil FR,"1:23.29,3",False,"0:5.33,0",sam. 18.03.2000,Kerzerslauf,15.0421


In [279]:
gender_names=pd.read_csv('gender_names.csv')

In [280]:
del gender_names['Unnamed: 0']

In [281]:
gender_names=gender_names.set_index(gender_names.name)

In [282]:
del gender_names['name']

In [283]:
gender_names.head(2)

,Female,Male,male_prob,female_prob
name,,,,
aaban,0,87,100.0,0.0
aabha,28,0,0.0,100.0


### Category from sex database

In [47]:
category_list=list(df.cat.value_counts().index)

In [53]:
len(category_list)

4639

In [48]:
category_dict_sex={}

In [49]:
cat_to_check=['B','D','F40','F/3','H','12','10-F20']

In [50]:
for i,cat in enumerate(category_list):
    if cat in category_dict_sex:
        continue
    names=list(df[df.cat==cat].nom.values)
    not_found=0
    male=0
    female=0
    for index,name in enumerate(names):
        splitted=re.split("[- ]+",name)
        found=0
        for name in splitted:
            try:
                row=gender_names.loc[name.lower()]
                male+=row.male_prob
                female+=row.female_prob
                found+=1
            except:
                pass
    if male!=0 or female!=0:
        male_perc=male/(male+female)
        category_dict_sex[cat]=male_perc
        if male_perc<0.85 and male_perc>0.15:
            if cat not in cat_to_check:
                cat_to_check.append(cat)
#             print('Category',cat,'Perc:',male_perc)
    if i%100==0:
#         print()
        print(i)
#         print()
    if i>7000:
        break

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600


In [51]:
len(cat_to_check)

1699

In [56]:
len(category_dict_sex)

4612

In [55]:
cat_parsed=0
for cat in category_dict_sex:
    if category_dict_sex[cat]>0.85:
        cat_parsed+=1
        category_dict_sex[cat]='M'
    elif category_dict_sex[cat]<0.15:
        cat_parsed+=1
        category_dict_sex[cat]='F'

In [58]:
category_dict_sex_copy=copy.deepcopy(category_dict_sex)

### Improvements

In [107]:
category_dict_sex=copy.deepcopy(category_dict_sex_copy)

In [108]:
to_exclude=['P-W','D','SM2/D','W','P/W']

In [109]:
cat_parsed=0
for cat in category_dict_sex:
    if cat in to_exclude:
        continue
    if category_dict_sex[cat]=='M' or category_dict_sex[cat]=='F':
        cat_parsed+=1
        continue
    cat_splitted=re.split("[-/]+",cat)
    for i in cat_splitted:
        if len(i)==3 and (i[0]=='M' or i[0]=='H') and i[1:].isdigit():
#             print(cat,category_dict_sex[cat])
            category_dict_sex[cat]='M'
            cat_parsed+=1
            break
        elif len(i)==3 and (i[0]=='F' or i[0]=='W')  and i[1:].isdigit():
#             print(cat,category_dict_sex[cat])
            category_dict_sex[cat]='F'
            cat_parsed+=1
            break
        if 'Fille' in i or 'Women' in i or 'Dam' in i:
            print(cat,category_dict_sex[cat])
            category_dict_sex[cat]='F'
            cat_parsed+=1
            break
        if 'Gar' in i or 'Men' in i or 'Hom' in i: 
            print(cat,category_dict_sex[cat])
            category_dict_sex[cat]='M'
            cat_parsed+=1
            break
        if i=='W' or i=='D' or i=='H-U20W' or i=='H-WJ':
            print(cat,category_dict_sex[cat])
            category_dict_sex[cat]='F'
            cat_parsed+=1
            break
cat_parsed

Hommes5 0.839018415668
C-D-U20 0.3023
W/K 0.40322
Dam1 0.185154219949
U16-W 0.18569047619
DamSen1 0.150454607509
C-W 0.275290909091
DamSen2 0.201291608392
DamSen 0.188779282869
D-Sen3 0.19718919598
M-D 0.173333333333
Duc-Hom4 0.842124166667
Damen2 0.178763190184
Hom V2 0.816015467626
5-Hom4 0.841405263158
W-F20 0.222911111111
Dam 0.190420140515
M-Fille4 0.173273553719
Damen4 0.164897222222
Hom III 0.827997560976
M-Fille2 0.170937297297
Hom5 0.790160869565
Dam4 0.2266703125
Hom V3 0.752517054264
Dam2 0.181056953642
O-Garc1 0.838885714286
21-Hom 0.818534615385
HomSen3 0.839172177419
Hand-Hom 0.660115
5-Hom3 0.795029213483
Hommes4 0.845066572724
DamSen3 0.186541304348
Garc-2 0.812878947368
Dam3 0.215437818182
Dam5 0.207342105263
S-Men 0.844298275862
Duc-Hom5 0.799691860465
10-Hom4 0.796443220339
M-Fille5 0.200543103448
D-Sen1 0.206074559471
Duc-Hom6 0.782147368421
10-Hom3 0.848979302832
Damen3 0.186778125
M-Men 0.82611
H-D 0.205403389831
Sch-D-M 0.25005
M-Fille1 0.167812626263
7-Dam2 0.18

3323

In [106]:
df[df.cat=='Sch-D-M']

,cat,sex,rang,nom,an,lieu,temps,retard,pace,Date,Name,Distance
index,,,,,,,,,,,,
1939913,Sch-D-M,M,12,Bauer Svenja,2002,Hochdorf,"0:4.48,1","0:0.56,3","0:4.48,0",dim. 28.08.2011,Run&Walk; Event Rothenburg,1
1939933,Sch-D-M,M,1,Birrer Alina,2001,Menznau,"0:3.51,8",False,"0:3.51,0",dim. 28.08.2011,Run&Walk; Event Rothenburg,1.00003
1939976,Sch-D-M,M,9,Büchler Natalie,2001,Rothenburg,"0:4.28,1","0:0.36,3","0:4.28,0",dim. 28.08.2011,Run&Walk; Event Rothenburg,1
1940054,Sch-D-M,M,2,Emmenegger Ana,2002,Nottwil,"0:4.4,1","0:0.12,3","0:4.4,0",dim. 28.08.2011,Run&Walk; Event Rothenburg,1
1940307,Sch-D-M,M,11,Kuster Alina,2001,Eschenbach LU,"0:4.44,7","0:0.52,9","0:4.44,0",dim. 28.08.2011,Run&Walk; Event Rothenburg,1.00002
1940314,Sch-D-M,M,6,"Lang Lorena, Rothenburg",2002,TSV Rothenburg,"0:4.22,9","0:0.31,1","0:4.22,0",dim. 28.08.2011,Run&Walk; Event Rothenburg,1.00003
1940443,Sch-D-M,M,3,"Oberli Chievi, Rothenburg",2001,TSV Rothenburg,"0:4.8,6","0:0.16,8","0:4.8,0",dim. 28.08.2011,Run&Walk; Event Rothenburg,1.00002
1940564,Sch-D-M,M,7,Schubert Julia,2001,Ebikon,"0:4.23,0","0:0.31,2","0:4.23,0",dim. 28.08.2011,Run&Walk; Event Rothenburg,1
1940588,Sch-D-M,M,10,Sigrist Anouk,2002,Luzern,"0:4.29,6","0:0.37,8","0:4.29,0",dim. 28.08.2011,Run&Walk; Event Rothenburg,1.00002


### Final classification - accurate

In [287]:
del df['sex']

In [289]:
def add_sex(x):
    cat=x.cat.iloc[0]
    try:
        sex=category_dict_sex[cat]
        if sex=='M' or sex=='F':
            x['sex']=sex
        else:
            x['sex']=False
    except:
        x['sex']=False
    return x

In [290]:
grouped=df.groupby(df.cat)

In [291]:
df=grouped.apply(add_sex)

In [292]:
df.sex.value_counts()

M        1194370
False     693113
F         457054
Name: sex, dtype: int64

In [293]:
df_sex=df[df.sex!=False]

In [294]:
df_sex.head(2)

,cat,rang,nom,an,lieu,temps,retard,pace,Date,Name,Distance,sex
index,,,,,,,,,,,,
3027,M20,233,Abgottspon Peter,74,Zermatt,"1:2.25,0",False,"0:4.9,0",sam. 18.03.2000,Kerzerslauf,15.0402,M
3028,M35,32,Abplanalp Michael,64,Bern,"0:55.11,7",False,"0:3.40,0",sam. 18.03.2000,Kerzerslauf,15.05,M


# Parse year (roughly eliminating clearly bad entries)

In [295]:
df.an=df.an.apply(lambda x: False if (x=='??' or x=='????' or x=='xxxx')  else x)

In [296]:
df.an=df.an.apply(lambda x: int(x))

In [297]:
df=df[df.an<2017]

In [298]:
df=df[(df.an>1870) | (df.an<100)]

# Columns names

In [299]:
df.head(1)

,cat,rang,nom,an,lieu,temps,retard,pace,Date,Name,Distance,sex
index,,,,,,,,,,,,
3027,M20,233,Abgottspon Peter,74,Zermatt,"1:2.25,0",False,"0:4.9,0",sam. 18.03.2000,Kerzerslauf,15.0402,M


In [300]:
df.columns.values

array(['cat', 'rang', 'nom', 'an', 'lieu', 'temps', 'retard', 'pace',
       'Date', 'Name', 'Distance', 'sex'], dtype=object)

In [301]:
df=df[['Name','Date','Distance','nom','sex','an','lieu','rang','temps','retard','pace']]

In [302]:
df.head()

,Name,Date,Distance,nom,sex,an,lieu,rang,temps,retard,pace
index,,,,,,,,,,,
3027,Kerzerslauf,sam. 18.03.2000,15.0402,Abgottspon Peter,M,74,Zermatt,233,"1:2.25,0",False,"0:4.9,0"
3028,Kerzerslauf,sam. 18.03.2000,15.05,Abplanalp Michael,M,64,Bern,32,"0:55.11,7",False,"0:3.40,0"
3029,Kerzerslauf,sam. 18.03.2000,15.0414,Abt Werner,M,47,Spiez,155,"1:12.42,9",False,"0:4.50,0"
3030,Kerzerslauf,sam. 18.03.2000,15.0182,Ackermann Antoinette,F,53,Alterswil,48,"1:22.36,7",False,"0:5.30,0"
3031,Kerzerslauf,sam. 18.03.2000,15.0421,Ackermann Hedy,F,46,Alterswil FR,42,"1:23.29,3",False,"0:5.33,0"


In [303]:
df.columns=['Race','Date','Distance','Name','Sex','Year','LivingPlace','Rank','Time','Delay','Pace']

In [304]:
df=df.reset_index()

In [305]:
del df['index']

In [316]:
df.head(2)

,Race,Date,Distance,Name,Sex,Year,LivingPlace,Rank,Time,Delay,Pace,round
0,Kerzerslauf,sam. 18.03.2000,15.0,Abgottspon Peter,M,74,Zermatt,233,"1:2.25,0",False,"0:4.9,0",15.0
1,Kerzerslauf,sam. 18.03.2000,15.0,Abplanalp Michael,M,64,Bern,32,"0:55.11,7",False,"0:3.40,0",15.0


# Distance

In [307]:
df.Distance=df.Distance.apply(lambda x: float(x))

In [308]:
df['round']=df.Distance.round()

In [309]:
df.Distance=df.Distance.apply(lambda x: x if (abs(x-round(x)))>0.065 else round(x))

In [310]:
df.Distance=df.Distance.apply(lambda x: 21.0975 if (abs(x-21.0975))<0.1 else x)

In [313]:
df.Distance=df.Distance.apply(lambda x: 42.195 if (abs(x-42.195))<0.1 else x)

In [314]:
df.Distance.value_counts()

10.000000     260645
21.097500     248291
5.000000      138138
42.195000     126106
15.000000      72747
1.000000       42800
4.000000       40216
20.000000      37429
2.000000       33507
11.000000      29572
12.000000      10884
8.000000       10765
17.000000       6500
31.000000       5433
30.000000       5290
9.000000        4877
3.000000        4699
6.000000        4223
26.000000       3854
27.000000       3295
100.000000      2984
7.000000        2611
35.000000       1822
14.000000       1590
25.000000       1307
0.500000        1301
51.000000       1015
19.000000        894
5.500000         796
16.000000        748
               ...  
8.668457           1
8.869219           1
18.503452          1
7.512385           1
1.200395           1
2.399370           1
3.399370           1
16.116716          1
4.796729           1
1.448293           1
7.819285           1
0.198475           1
1.311579           1
8.515155           1
8.178270           1
42.297646          1
2.198293     

In [317]:
del df['round']

In [318]:
df.head(5)

,Race,Date,Distance,Name,Sex,Year,LivingPlace,Rank,Time,Delay,Pace
0,Kerzerslauf,sam. 18.03.2000,15.0,Abgottspon Peter,M,74,Zermatt,233,"1:2.25,0",False,"0:4.9,0"
1,Kerzerslauf,sam. 18.03.2000,15.0,Abplanalp Michael,M,64,Bern,32,"0:55.11,7",False,"0:3.40,0"
2,Kerzerslauf,sam. 18.03.2000,15.0,Abt Werner,M,47,Spiez,155,"1:12.42,9",False,"0:4.50,0"
3,Kerzerslauf,sam. 18.03.2000,15.0,Ackermann Antoinette,F,53,Alterswil,48,"1:22.36,7",False,"0:5.30,0"
4,Kerzerslauf,sam. 18.03.2000,15.0,Ackermann Hedy,F,46,Alterswil FR,42,"1:23.29,3",False,"0:5.33,0"


# Year - trying to get better data

In [340]:
df=df[df.Sex!=False]

In [341]:
df.Year.value_counts().index.values

array([1968, 1967, 1966, 1969, 1965, 1970, 1964, 1971, 1963, 1972, 1973,
       1962, 1974, 1975, 1976, 1978, 1977, 1961, 1979, 1980, 1981, 1982,
       1960, 1983, 1984, 1959, 1985, 1986, 2002, 2003, 2001, 2000, 1958,
       2004, 1999, 1957, 1987, 1988, 1998, 1956, 2005, 1989, 1997, 1990,
       2006,   63, 1955,   64, 1996,   62, 1991,   60,   61,   65, 1954,
         66, 1995, 1993, 1992,   59, 1994, 1953,   67,   58,   57,   68,
       1952,   56, 2007,   69, 1951,   70,   71,   72, 1950,   55,   73,
         54,   94,   95, 2008,   53, 1949,   74,   76,   75,   96,   93,
         52, 1948, 1947,   92,   78,   97,   77,   51,    0, 1946,   91,
         79,   50, 2009, 1945,   80,   49,   98,   81, 1944,   48,   82,
         90,   47,   89,   46, 1943,   83,   45,   88,   84, 1942,   99,
       2010,   86,   44, 1941,   85,   43,   87,   42,   41, 1940, 1939,
       1938,   40, 1937, 2011,   38,   39,   37, 1936,   36,    1, 1934,
         35, 1935,   34, 2012,   33,   32, 1932,   

In [342]:
df.Year=df.Year.apply(lambda x: 0 if x==2 else x)

/home/stefano/anaconda3/lib/python3.5/site-packages/pandas/core/generic.py:2701: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [343]:
df[df.Year>1900].shape

(1281195, 11)

In [344]:
df[df.Year<1900].shape

(370073, 11)

# Output
First output

In [345]:
df_output=df[df.Year>1900]

In [347]:
df_output.shape

(1281195, 11)

In [362]:
df_output.head()

,Race,Date,Distance,Name,Sex,Year,LivingPlace,Rank,Time,Delay,Pace,len_name
256022,"Jungfrau-Marathon, Interlaken",sam. 06.09.2003,42.195000,Abächerli Walter,M,1952,Hinwil,47,"4:31.56,1","0:53.5,3","0:6.26,0",3
256023,"Jungfrau-Marathon, Interlaken",sam. 06.09.2003,42.195000,Abbringh Ellen,F,1962,NL-Doorn 3941 EB,91,"5:55.9,4","2:12.11,6","0:8.25,0",3
256024,"Jungfrau-Marathon, Interlaken",sam. 06.09.2003,42.195000,Abegglen Eddy,M,1954,Mürren,424,"5:45.21,9","2:20.33,8","0:8.11,0",3
256025,"Jungfrau-Marathon, Interlaken",sam. 06.09.2003,42.328675,Abosa Emebet,F,1974,Zuoz,1,"3:21.46,1",False,"0:4.46,0",3
256026,"Jungfrau-Marathon, Interlaken",sam. 06.09.2003,42.195000,Abplanalp Michel,M,1960,Auvernier,143,"4:30.26,9","1:7.35,3","0:6.24,0",3


In [369]:
df_output['Name']=df_output.Name.apply(lambda x: x.split(',')[0])

/home/stefano/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [370]:
df_output['len_name']=df_output.Name.apply(lambda x: len(x.rstrip().lstrip().split(' ')))

/home/stefano/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [376]:
df_output['Name']=df_output.Name.apply(lambda x: x.split('+')[0])

/home/stefano/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [377]:
df_output

,Race,Date,Distance,Name,Sex,Year,LivingPlace,Rank,Time,Delay,Pace,len_name
256022,"Jungfrau-Marathon, Interlaken",sam. 06.09.2003,42.195000,Abächerli Walter,M,1952,Hinwil,47,"4:31.56,1","0:53.5,3","0:6.26,0",2
256023,"Jungfrau-Marathon, Interlaken",sam. 06.09.2003,42.195000,Abbringh Ellen,F,1962,NL-Doorn 3941 EB,91,"5:55.9,4","2:12.11,6","0:8.25,0",2
256024,"Jungfrau-Marathon, Interlaken",sam. 06.09.2003,42.195000,Abegglen Eddy,M,1954,Mürren,424,"5:45.21,9","2:20.33,8","0:8.11,0",2
256025,"Jungfrau-Marathon, Interlaken",sam. 06.09.2003,42.328675,Abosa Emebet,F,1974,Zuoz,1,"3:21.46,1",False,"0:4.46,0",2
256026,"Jungfrau-Marathon, Interlaken",sam. 06.09.2003,42.195000,Abplanalp Michel,M,1960,Auvernier,143,"4:30.26,9","1:7.35,3","0:6.24,0",2
256027,"Jungfrau-Marathon, Interlaken",sam. 06.09.2003,42.195000,Achermann Erich,M,1968,Buochs,413,"4:46.29,7","1:57.28,1","0:6.47,0",2
256028,"Jungfrau-Marathon, Interlaken",sam. 06.09.2003,42.195000,Achten Jan,M,1943,NL-CG Amsterdam,63,"5:37.59,7","1:25.46,8","0:8.0,0",2
256029,"Jungfrau-Marathon, Interlaken",sam. 06.09.2003,42.195000,Achterberg Nico,M,1957,NL-Veenendaal 3902 DG,204,"4:56.59,0","1:32.10,9","0:7.2,0",2
256030,"Jungfrau-Marathon, Interlaken",sam. 06.09.2003,42.195000,Ackermann Antoinette,F,1953,Alterswil FR,40,"6:13.10,7","1:46.28,7","0:8.50,0",2
256031,"Jungfrau-Marathon, Interlaken",sam. 06.09.2003,42.195000,Adam Felix,M,1958,Erlenbach,324,"5:18.39,5","1:53.51,4","0:7.33,0",2


In [378]:
df_output.len_name.value_counts()

2    1227920
3      47636
4       5078
5        370
1        179
6         12
Name: len_name, dtype: int64

In [381]:
df_output[df_output.len_name==1]

,Race,Date,Distance,Name,Sex,Year,LivingPlace,Rank,Time,Delay,Pace,len_name
468472,"Jungfrau-Marathon, Interlaken",sam. 10.09.2005,42.195000,Kollerjosef,M,1965,Speicher,219,"4:43.13,3","1:32.35,2","0:6.42,0",1
469963,"Jungfrau-Marathon, Interlaken",sam. 10.09.2005,42.195000,Tomas,M,1971,GB-1eb BL8,329,"4:37.36,5","1:38.14,7","0:6.34,0",1
542553,20km de Lausanne,sam. 22.04.2006,20.000000,Rollinson,M,1962,Perroy,43,"1:25.50,0","0:20.42,0","0:4.17,0",1
687866,"Zürich Marathon, Teamrun und Cityrun",dim. 01.04.2007,42.299045,Andrea,F,1972,Zürich,59,"3:39.15,3","0:52.29,1","0:5.11,0",1
693249,Pfingstlauf Wohlen AG,sam. 26.05.2007,0.350202,Widrig,M,2003,Hittnau,49,"0:2.53,0","0:1.34,6","0:8.14,0",1
693250,Pfingstlauf Wohlen AG,sam. 26.05.2007,1.601929,Widrig,M,1997,Hittnau,4,"0:5.38,7","0:0.14,4","0:3.31,0",1
694285,Winterthur Marathon,dim. 20.05.2007,21.097500,Leuppisamir,M,1993,Winterthur,48,"2:5.3,9","0:44.52,1","0:5.57,0",1
695051,Winterthur Marathon,dim. 20.05.2007,42.195000,Vauclairjean-Marie,M,1965,Moutier,23,"3:31.34,2","0:47.51,1","0:5.1,0",1
695525,"Dreiländer-Lauf, Basel",dim. 20.05.2007,10.000000,Hegnauer,M,1977,Basel,8,"0:43.43,5","0:7.8,5","0:4.22,0",1
695943,"Dreiländer-Lauf, Basel",dim. 20.05.2007,21.097500,Wikmann,M,1956,VC Weil,5,"1:31.29,9","0:12.7,2","0:4.20,0",1


In [382]:
df_output.to_csv('full_database.csv',index=False)